In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns

In [5]:
clients = pd.read_excel('CASE_CLIENTS.xlsx')

In [6]:
contracts = pd.read_excel('CASE_CONTRACTS.xlsx')

In [9]:
losses = pd.read_excel('CASE_LOSSES.xlsx')

In [10]:
experiment = pd.read_excel('experiment_group.xlsx')

# Data preparation

In [11]:
clients.head()

,client_id,last_name,first_name,middle_name,age,sex
0,10000041307,З*****,Р*****,О*****,30,F
1,10000133158,И*****,М*****,Ю*****,37,M
2,100003773,Е*****,О*****,С*****,50,M
3,10000535718,Е*****,Ф*****,Ч*****,45,M
4,10000583119,И*****,Ф*****,Ф*****,28,M


In [26]:
contracts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3711 entries, 0 to 3710
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   contract_id       3711 non-null   int64 
 1   contract_num      3711 non-null   object
 2   product_name      3711 non-null   object
 3   client_id         3711 non-null   int64 
 4   contract_status   3711 non-null   object
 5   currency_name     3711 non-null   object
 6   duration          3711 non-null   int64 
 7   country           3711 non-null   object
 8   price             3711 non-null   int64 
 9   insurance_amount  3711 non-null   int64 
dtypes: int64(5), object(5)
memory usage: 290.1+ KB


In [13]:
losses.head()

,loss_id,client_id,loss_name,loss_payout_amt
0,1,10000713193,Оказание медицинской помощи,100000
1,2,100014656,Оказание медицинской помощи,50000
2,3,10006545463,Оказание медицинской помощи,30000
3,4,10006611194,Оказание медицинской помощи,70000
4,5,1001091597,Оказание медицинской помощи,60000


In [28]:
experiment

,client_id,group,experiment_id
0,10001718,control,policy_pricing
1,1017180395,control,policy_pricing
2,1017178047,control,policy_pricing
3,1017174614,control,policy_pricing
4,1017174071,control,policy_pricing
...,...,...,...
14837,11919548695,test,policy_pricing
14838,11919548696,test,policy_pricing
14839,11996903168,test,policy_pricing
14840,11996903169,test,policy_pricing


In [38]:
experiment.head()

,client_id,group,experiment_id
0,10001718,control,policy_pricing
1,1017180395,control,policy_pricing
2,1017178047,control,policy_pricing
3,1017174614,control,policy_pricing
4,1017174071,control,policy_pricing


In [39]:
experiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14842 entries, 0 to 14841
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   client_id      14842 non-null  int64 
 1   group          12349 non-null  object
 2   experiment_id  14842 non-null  object
dtypes: int64(1), object(2)
memory usage: 348.0+ KB


In [24]:
experiment.group.unique()

array(['control', 'test', nan], dtype=object)

In [27]:
experiment.group.isna().sum()

2493

In [29]:
experiment[experiment.group.isna()]

,client_id,group,experiment_id
11729,10113558690,NaN,policy_pricing
11730,10113558691,NaN,policy_pricing
11731,10113608059,NaN,policy_pricing
11732,10113608060,NaN,policy_pricing
11733,10113608061,NaN,policy_pricing
...,...,...,...
14217,10171096014,NaN,policy_pricing
14218,10171096015,NaN,policy_pricing
14219,10171096016,NaN,policy_pricing
14220,10171110196,NaN,policy_pricing


In [30]:
experiment.group.value_counts()

,count
group,
test,6306
control,6043


Удаляем строки где группа не указана.

In [32]:
final_exp = experiment.copy()
final_exp.dropna(inplace=True)
final_exp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12349 entries, 0 to 14841
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   client_id      12349 non-null  int64 
 1   group          12349 non-null  object
 2   experiment_id  12349 non-null  object
dtypes: int64(1), object(2)
memory usage: 385.9+ KB


In [37]:
final_exp.group.value_counts()

,count
group,
test,6306
control,6043


In [36]:
final_exp.experiment_id.value_counts()

,count
experiment_id,
policy_pricing,12349


In [35]:
final_exp.client_id.nunique()

12349

In [40]:
merged_df = contracts.merge(clients, on='client_id', how='left')
merged_df = merged_df.merge(losses, on='client_id', how='left')
merged_df = merged_df.merge(final_exp, on='client_id', how='left')

In [41]:
merged_df.head()

,contract_id,contract_num,product_name,client_id,contract_status,currency_name,duration,country,price,insurance_amount,last_name,first_name,middle_name,age,sex,loss_id,loss_name,loss_payout_amt,group,experiment_id
0,21111219,ТТЕ7227715*****,Страхование путешественников,10161870404,Действует,Российский рубль,10,Беларусь,1096,2000000,А*****,Х*****,А*****,20,M,NaN,NaN,NaN,test,policy_pricing
1,21111381,ТТЕ7227715*****,Страхование путешественников,1017159879,Действует,Российский рубль,7,Индонезия,1918,5000000,З*****,Х*****,М*****,55,M,NaN,NaN,NaN,control,policy_pricing
2,21112353,БАДАМСТЕ55*****,Страхование путешественников,10161883357,Действует,Российский рубль,10,Беларусь,1096,2000000,Б*****,В*****,Щ*****,61,F,NaN,NaN,NaN,control,policy_pricing
3,21113224,ТТЕ7227715*****,Страхование путешественников,100766461,Действует,Российский рубль,14,Грузия,1534,2000000,И*****,У*****,Н*****,25,M,NaN,NaN,NaN,control,policy_pricing
4,21113292,ТТЕ7227715*****,Страхование путешественников,10171604758,Действует,Российский рубль,7,Аргентина,1918,5000000,З*****,Ч*****,Г*****,33,M,NaN,NaN,NaN,control,policy_pricing


In [65]:
# Разделяем на контрольную и тестовую группы
control_group = merged_df[merged_df['group'] == 'control']
test_group = merged_df[merged_df['group'] == 'test']


In [66]:
control_group.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1949 entries, 1 to 3710
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   contract_id       1949 non-null   int64  
 1   contract_num      1949 non-null   object 
 2   product_name      1949 non-null   object 
 3   client_id         1949 non-null   int64  
 4   contract_status   1949 non-null   object 
 5   currency_name     1949 non-null   object 
 6   duration          1949 non-null   int64  
 7   country           1949 non-null   object 
 8   price             1949 non-null   int64  
 9   insurance_amount  1949 non-null   int64  
 10  last_name         1949 non-null   object 
 11  first_name        1938 non-null   object 
 12  middle_name       1804 non-null   object 
 13  age               1949 non-null   int64  
 14  sex               1949 non-null   object 
 15  loss_id           30 non-null     float64
 16  loss_name         30 non-null     object 
 17  

In [67]:
test_group.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1667 entries, 0 to 3708
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   contract_id       1667 non-null   int64  
 1   contract_num      1667 non-null   object 
 2   product_name      1667 non-null   object 
 3   client_id         1667 non-null   int64  
 4   contract_status   1667 non-null   object 
 5   currency_name     1667 non-null   object 
 6   duration          1667 non-null   int64  
 7   country           1667 non-null   object 
 8   price             1667 non-null   int64  
 9   insurance_amount  1667 non-null   int64  
 10  last_name         1667 non-null   object 
 11  first_name        1660 non-null   object 
 12  middle_name       1562 non-null   object 
 13  age               1667 non-null   int64  
 14  sex               1667 non-null   object 
 15  loss_id           14 non-null     float64
 16  loss_name         14 non-null     object 
 17  

In [68]:
test_group.isna().sum()

,0
contract_id,0
contract_num,0
product_name,0
client_id,0
contract_status,0
currency_name,0
duration,0
country,0
price,0
insurance_amount,0


# Checking the metrics

Метрики для теста
- Средняя цена полиса (price).
- Конверсия в оформление (доля успешных контрактов).
- Средняя убыточность (loss_payout_amt).

In [69]:
control_group.contract_status.value_counts()

,count
contract_status,
Действует,1882
Завершен,67


In [86]:
control_group.head()

,contract_id,contract_num,product_name,client_id,contract_status,currency_name,duration,country,price,insurance_amount,last_name,first_name,middle_name,age,sex,loss_id,loss_name,loss_payout_amt,group,experiment_id
1,21111381,ТТЕ7227715*****,Страхование путешественников,1017159879,Действует,Российский рубль,7,Индонезия,1918,5000000,З*****,Х*****,М*****,55,M,NaN,NaN,NaN,control,policy_pricing
2,21112353,БАДАМСТЕ55*****,Страхование путешественников,10161883357,Действует,Российский рубль,10,Беларусь,1096,2000000,Б*****,В*****,Щ*****,61,F,NaN,NaN,NaN,control,policy_pricing
3,21113224,ТТЕ7227715*****,Страхование путешественников,100766461,Действует,Российский рубль,14,Грузия,1534,2000000,И*****,У*****,Н*****,25,M,NaN,NaN,NaN,control,policy_pricing
4,21113292,ТТЕ7227715*****,Страхование путешественников,10171604758,Действует,Российский рубль,7,Аргентина,1918,5000000,З*****,Ч*****,Г*****,33,M,NaN,NaN,NaN,control,policy_pricing
5,21114111,ДСЖ-5/2771*****,Страхование путешественников,100651616,Действует,Российский рубль,30,Турция,4932,3000000,Д*****,В*****,-*****,33,F,NaN,NaN,NaN,control,policy_pricing


In [71]:
control_group.shape

(1949, 20)

In [75]:
control_group[control_group['contract_status'] == 'Действует'].shape[0]

(1882, 20)

In [93]:
metrics = {
    'mean_price': [control_group['price'].mean(), test_group['price'].mean(), control_group['price'].mean()/test_group['price'].mean() * 100],
    'conversion': [
        control_group[control_group['contract_status'] == 'Действует'].shape[0]/control_group.shape[0],
        (test_group[test_group['contract_status'] == 'Действует'].shape[0]/test_group.shape[0]),
        (control_group[control_group['contract_status'] == 'Действует'].shape[0]/control_group.shape[0]) - (test_group[test_group['contract_status'] == 'Действует'].shape[0]/test_group.shape[0])
    ],
    'mean_loss': [control_group['loss_payout_amt'].mean(), test_group['loss_payout_amt'].mean(), control_group['loss_payout_amt'].mean()/test_group['loss_payout_amt'].mean() * 100],
    'mean_insurance': [control_group['insurance_amount'].mean(), test_group['insurance_amount'].mean(), control_group['insurance_amount'].mean()/test_group['insurance_amount'].mean() * 100]
}
metrics_df = pd.DataFrame(metrics, index=['Control', 'Test', 'Percent'])

# t-тест для значимости различий
price_ttest = stats.ttest_ind(control_group['price'], test_group['price'])
conversion_ttest = stats.ttest_ind(
    (control_group['contract_status'] == 'Действует').astype(int),
    (test_group['contract_status'] == 'Действует').astype(int)
)
loss_ttest = stats.ttest_ind(control_group['loss_payout_amt'].fillna(0), test_group['loss_payout_amt'].fillna(0))
ins_ttest = stats.ttest_ind(control_group['insurance_amount'].fillna(0), test_group['insurance_amount'].fillna(0))

print("Средние значения метрик:")
print(metrics_df)
print("\nT-test results:")
print(f"Price p-value: {price_ttest.pvalue}")
print(f"Conversion p-value: {conversion_ttest.pvalue}")
print(f"Loss p-value: {loss_ttest.pvalue}")
print(f"Insurance p-value: {ins_ttest.pvalue}")

Средние значения метрик:
          mean_price  conversion      mean_loss  mean_insurance
Control  3623.322217    0.965623  212666.666667    3.615854e+06
Test     4636.203359    0.802639  245714.285714    2.828314e+06
Percent    78.152789    0.162984      86.550388    1.278449e+02

T-test results:
Price p-value: 0.0016764343314087742
Conversion p-value: 5.0890381904346065e-57
Loss p-value: 0.43133270280569314
Insurance p-value: 2.2706998061194915e-57


In [89]:
ttests = {
    'Средняя цена полиса': price_ttest.pvalue,
    'Конверсия на оформление полиса': conversion_ttest.pvalue,
    'Средняя цена по убыткам ': loss_ttest.pvalue,
    'Средняя цена по Страховой сумме': ins_ttest.pvalue
}


for key, value in ttests.items():
  if value < 0.05:
      print(f"{key} отличается.")
  else:
      print(f"{key} не отличается.")

Средняя цена полиса отличается.
Конверсия на оформление полиса отличается.
Средняя цена по убыткам  не отличается.
Средняя цена по Страховой сумме отличается.


# **Бизнес-рекомендация по результатам A/B-теста**  

**Выводы по метрикам:**  
1. **Цена полиса (mean_price) выросла** в тестовой группе (**22%**), что может привести к увеличению доходов.  
   - p-value = **0.0017** (разница статистически значима).  

2. **Конверсия (conversion) снизилась** в тестовой группе (**-16,9%**), что может означать отток клиентов из-за повышения цен.  
   - p-value = **5.08e-57** (разница статистически значима).  

3. **Средняя убыточность (mean_loss) не изменилась** – это хороший знак, так как клиенты с разными ценами полиса несут схожие убытки.  
   - p-value = **0.4313** (разница **не** значима).  

4. **Средняя страховая сумма (mean_insurance) снизилась** (**-21,8%**) в тестовой группе. Это значит, что клиенты стали страховаться на меньшие суммы.  
   - p-value = **2.27e-57** (разница статистически значима).  

---

**Рекомендация для бизнеса**  

**Позитивное влияние тестового подхода:**  
**Рост средней цены полиса** - потенциально **увеличение выручки**.  
**Снижение страховой суммы** - потенциально **снижение страховых выплат**.  
**Убыточность осталась стабильной**, несмотря на изменение стратегии ценообразования.  

**Негативное влияние:**  
**Сильное падение конверсии** → часть клиентов **отказывается** от покупки из-за высокой цены.  
Возможный отток клиентов к конкурентам.  

---

**Окончательное решение:**  
**Тестовый подход приводит к снижению количества клиентов, но может быть выгоден за счёт роста цены полиса. Предлагаю не вводить новый метод формированию стоимости полиса ВЗР, так как это приведет к снижению количества клиентов, что скажется на прибыли.**  
**Рекомендация:** провести новый A/B-тест с несильным повышением стоимости полиса!